In [1]:
import pandas as pd
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser 
from langchain.chains import LLMChain

In [2]:
clean_data = pd.read_csv("clean_data.csv")
clean_data.head()

,Type,Post_ID,Title,Author,Timestamp,Text,Score,Total_Comments,Post_URL,Filtered_Verdict,Verdict,Popular,clean_text
0,Post,1grc8a6,AITA for not giving my late husband’s mom any ...,Icy-Sun6781,2024-11-14 18:56:10,My first and hopefully last throwaway account....,7253,2931,https://www.reddit.com/r/AmItheAsshole/comment...,The final verdict is: **Everyone Sucks**\n\nJu...,NaN,ESH,My first and hopefully last throwaway account....
1,Comment,1grc8a6,AITA for not giving my late husband’s mom any ...,Tdluxon,2024-11-14 19:03:08,ESH\n\nWhen you found out that you were going ...,12198,0,NaN,NaN,ESH,NaN,ESH When you found out that you were going to ...
2,Comment,1grc8a6,AITA for not giving my late husband’s mom any ...,Extra-Visit-8385,2024-11-14 19:27:31,"NTA. As a daughter, I paid for a large portion...",1180,0,NaN,NaN,NTA,NaN,"NTA. As a daughter, I paid for a large portion..."
3,Comment,1grc8a6,AITA for not giving my late husband’s mom any ...,JuggernautWilling851,2024-11-14 20:06:29,I don’t get al the Y T A votes. So many people...,816,0,NaN,NaN,NTA,NaN,I don’t get al the Y T A votes. So many people...
4,Comment,1grc8a6,AITA for not giving my late husband’s mom any ...,junkdumper,2024-11-14 19:56:05,NTA\n\nShe offered to pay for a funeral FOR HE...,493,0,NaN,NaN,NTA,NaN,NTA She offered to pay for a funeral FOR HER S...


In [3]:
slates = pd.read_json("viewpoints.json")
slates

,1gi2awo,1gi378c,1gi5k6b,1gi6rmt,1giwkq5,1giy3mj,1gj6zkq,1gjc9pm,1gjquni,1gjvwc2,1gjy9l8,1gjyyhu,1gk6tzm,1gkbmkz,1gkhfq7,1gkhp7b,1gkukkp
1,Boyfriend crossed a line by using a personal a...,Talk to the neighbor and ask them to stop idli...,Talk to the neighbor and ask them to stop idli...,She was too sick to meet family but well enoug...,She stole your car and held your stuff hostage.,The person making eggs should make an effort t...,You should give your grandmother notice so she...,You should give your grandmother notice so she...,You should give your grandmother notice so she...,You should give your grandmother notice so she...,Check the label before opening a package to av...,Family recipes are meant to be kept within the...,Redoing poor work helps children learn from mi...,He's playing games to avoid paying for meals a...,He's playing games to avoid paying for meals a...,He's playing games to avoid paying for meals a...,"Respect people's wishes and boundaries, even i..."
2,He should have known better than to use a dead...,Report the neighbor to the police for excessiv...,Report the neighbor to the police for excessiv...,Her accusation of cheating sounds like project...,The relationship is toxic and it's better that...,It's not normal to eat bits of eggshell in eggs.,You've been living with her virtually rent-fre...,You've been living with her virtually rent-fre...,You've been living with her virtually rent-fre...,You've been living with her virtually rent-fre...,"It's common sense to check the label, even if ...",Sharing recipes is a way to show love and appr...,The goal of education is not just to complete ...,You should have an open conversation about spl...,You should have an open conversation about spl...,You should have an open conversation about spl...,Going behind someone's back to do something th...
3,"The game's goal is to be controversial, but th...",Check local laws and ordinances regarding idli...,Check local laws and ordinances regarding idli...,You were petty and controlling by not letting ...,You could've called the police when she didn't...,The husband should crack his own eggs if he ha...,You're acting like a freeloader who's taking a...,You're acting like a freeloader who's taking a...,You're acting like a freeloader who's taking a...,You're acting like a freeloader who's taking a...,Marriage doesn't mean you have access to your ...,Gatekeeping recipes is petty and immature.,Forcing a child to redo an assignment can be s...,"It's reasonable to split costs 50/50, but his ...","It's reasonable to split costs 50/50, but his ...","It's reasonable to split costs 50/50, but his ...",Forcing someone to accept something they don't...
4,"Boyfriend's apology was insincere, as he impli...",Idling for 30 minutes is unnecessary and a was...,Idling for 30 minutes is unnecessary and a was...,She might have been nervous about meeting your...,She was trying to teach you a lesson by lying ...,"The issue is not about the eggs, but about the...",You should treat your grandmother like any oth...,You should treat your grandmother like any oth...,You should treat your grandmother like any oth...,You should treat your grandmother like any oth...,"If you're expecting a package, it's best to co...","Recipes are meant to be shared and enjoyed, no...",It's possible that the child didn't understand...,He's trying to see if you're willing to pay fo...,He's trying to see if you're willing to pay fo...,He's trying to see if you're willing to pay fo...,Sneaking money into someone's wallet is not a ...
5,OP should set boundaries and communicate what ...,"Older diesel trucks may need to warm up, but n...","Older diesel trucks may need to warm up, but n...",She should have communicated her feelings and ...,You should have zero contact with her at work ...,The person serving eggs should ensure they are...,You're being ungrateful and entitled for not a...,You're being ungrateful and entitled for not a...,You're being ungrateful and entitled fo

In [4]:
posts = list(slates.columns)

In [5]:
type(slates["1gi2awo"])

pandas.core.series.Series

In [6]:
AGENT_PROMPT_TEMPLATE = PromptTemplate.from_template( """


Given, the description and the comment as your stance, think like this person and try to order ALL the statements from the slate in an order that goes from MOST LIKED to LEAST LIKED.
A random example would be as follows :, if the example post description == "AITA for getting a rescue cat and giving it away later on because it was not my cup of tea?". 
Your example comment to this ==  "YTA, it was your duty to check and educate yourself about the responsibilites that come with adopting a pet cat and how sometimes they can be more difficult because they have been through abuse".
The example slate has 5 statements for example, 
1 : You are not the asshole here because you were doing a good deed but did not prepare for it.
2 : Well if it did not work out then it did not, no point in blaming people now.
3 : You are in the wrong here because you not only gave into your impulses but also were irresponsible with the duties.
4 : Cats look easy to maintain is not a good enough reason to get a stray animal as your pet. You are not prepared to get one
5 : You need to first intereact and see if you are a cat person at all. Reach out to people or friends who have pet cats. 

Given all this information you would have the following order of the statements and the reaasoning for the order.

ORDER = [3,1,4,5,2]
REASONING = Based on the comment you disagree and the poster is in the wrong. Logically, you agree with 1,3 but then also agree with 4,5 a little because that would mean being responsible. You do not agree with 2 so it goes last. 

Now that the example is done. Here are the inputs for the task that you have to perfom. You are a commentor on a post. The post is from r/AITA the with the following description: 
----
DESCRIPTION = {description} 
----
You have put the following comment on the post as a reflection of what you think : 
----
COMMENT = {comment}
----
These are the summarized statements/opinions across all the other comments(including yours) that you have to use for the ordering and give reasoning for that order.
----
SLATE = {slate}
----

RULES TO STRICTILY FOLLOW : 
1. DO NOT SKIP any statements.You NEED TO RANK all of the statements.
2. ONLY respond with ONE COMPLETE LIST OF ORDERING and the REASONING(based on your comment and slate) justifying that ordering.
3. FOLLOW the format from the example. You might have a different number of statements(10 0r 15) so ADAPT accordingly. 
4. Each RANK should be in the output EXACTLY ONCE.
5. Given above is JUST AN EXAMPLE for you. Do not use the same context, but try to understand how the task is to be done from this example.
6. DO NOT USE THE SAME SLATE or THE SAME ORDERING from the EXAMPLE, use the slate statements, description and comment given to you after the example for the task
7. DO NOT RETURN ANY EXTRA ITEMS except for the VALID ORDER and REASONING

""")

In [22]:
# new llm instance
llm = ChatGroq(
    model= "mixtral-8x7b-32768",
    groq_api_key = "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",
    temperature = 0
)

In [23]:
test = '1gi2awo'

description = clean_data.loc[
    (clean_data["Post_ID"] == test) & (clean_data["Type"] == "Post"), 
    "clean_text"
].values[0]

comments =  clean_data.loc[
    (clean_data["Post_ID"] == test) & (clean_data["Type"] != "Post"), 
    "clean_text"
]
slate = slates[test].dropna()
slate_dict = {num+1:value for num, value in enumerate(slate)}

In [24]:
# this is the comment data for each of the posts
comments_post_id = clean_data.loc[(clean_data["Post_ID"] == test)& (clean_data["Type"] != "Post")]

In [25]:
comments_post_id 

,Type,Post_ID,Title,Author,Timestamp,Text,Score,Total_Comments,Post_URL,Filtered_Verdict,Verdict,Popular,clean_text
2221,Comment,1gi2awo,AITA for snapping at my boyfriend after he tar...,GreekDudeYiannis,2024-11-02 18:34:21,NTA. I think there's a boundary between being ...,783,0,NaN,NaN,NTA,NaN,NTA. I think there's a boundary between being ...
2222,Comment,1gi2awo,AITA for snapping at my boyfriend after he tar...,Principessa116,2024-11-02 19:03:54,NTA. He deliberately crossed a line. \n\nHe th...,403,0,NaN,NaN,NTA,NaN,NTA. He deliberately crossed a line. He threw ...
2223,Comment,1gi2awo,AITA for snapping at my boyfriend after he tar...,veeunique,2024-11-02 18:38:59,"NTA, he admitted that he purposely used an imp...",69,0,NaN,NaN,NTA,NaN,"NTA, he admitted that he purposely used an imp..."
2224,Comment,1gi2awo,AITA for snapping at my boyfriend after he tar...,nerdcoffin,2024-11-02 18:20:28,"NTA. I know it's like, Cards Against Humanity ...",85,0,NaN,NaN,NTA,NaN,"NTA. I know it's like, Cards Against Humanity ..."
2225,Comment,1gi2awo,AITA for snapping at my boyfriend after he tar...,rotmonster,2024-11-02 18:34:03,The thing with games like cards against humani...,53,0,NaN,NaN,NaN,NaN,The thing with games like cards against humani...
2226,Comment,1gi2awo,AITA for snapping at my boyfriend after he tar...,Enough_Ad_222,2024-11-02 18:55:36,:( I don’t think you like being used for a win...,35,0,NaN,NaN,NTA,NaN,:( I don’t think you like being used for a win...
2227,Comment,1gi2awo,AITA for snapping at my boyfriend after he tar...,LawyerDad1981,2024-11-03 07:27:39,He pulled a totally dick move just to advance ...,4,0,NaN,NaN,NTA,NaN,He pulled a totally dick move just to advance ...
2228,Comment,1gi2awo,AITA for snapping at my boyfriend after he tar...,Dr_Drax,2024-11-02 19:18:01,Let's break down the end of your post:\n\n>He ...,40,0,NaN,NaN,NTA,NaN,Let's break down the end of your post: >He sai...
2229,Comment,1gi2awo,AITA for snapping at my boyfriend after he tar...,Fanoflif21,2024-11-02 19:08:38,NTA we've played that game (with our teenage k...,20,0,NaN,NaN,NTA,NaN,NTA we've played that game (with our teenage k...
2230,Comment,1gi2awo,AITA for snapping at my boyfriend after he tar...,Longjumping_Advice89,2024-11-02 18:31:43,NTA.\n\nThere are some things that you just do...,10,0,NaN,NaN,NTA,NaN,NTA. There are some things that you just don't...


In [26]:
slate_dict

{1: 'Boyfriend crossed a line by using a personal and sensitive topic to win a game.',
 2: 'He should have known better than to use a dead mom joke, even in a game meant to be offensive.',
 3: "The game's goal is to be controversial, but that doesn't mean it's okay to hurt someone's feelings.",
 4: "Boyfriend's apology was insincere, as he implied the OP was overreacting.",
 5: "OP should set boundaries and communicate what is and isn't okay to joke about.",
 6: "Boyfriend's actions show he's willing to hurt OP for personal gain, which is a red flag in the relationship.",
 7: "OP should consider whether boyfriend's behavior is a consistent pattern and if it's a sign of deeper issues.",
 8: "The game is meant to be uncomfortable, but that doesn't mean OP should just accept being hurt.",
 9: "Boyfriend's actions were not just about winning the game, but also about targeting OP specifically.",
 10: "OP's feelings are valid, and boyfriend should have been more understanding and empathetic.

In [27]:
comment = comments.values[20]
comment

'YTA, honestly. It sounds like you’re overreacting a bit. It’s just a game, and it was meant to be controversial. He probably didn’t think it would hit so close to home. If you can’t handle a little banter in a game, maybe you shouldn’t play. Plus, you said yourself you don’t remember your bio mom, so it seems like you’re making a bigger deal out of it than necessary. Just let it go; it’s not like he did it to hurt you on purpose.'

In [28]:
description

"I (F) was playing games with my boyfriend, and his friends. We played a game where the goal is to make a controversial prompt that the players of the game will be split on. The more split it is, the more points. Some more background about me, this is important. I was adopted by other relatives and have lived with them since I was 1. My bio mother is dead and has been for my entire life. I don't remember anything about her. I dont fully remember what the prompt was, it was something about getting money by having to delete the only picture of something you have off your phone. My boyfriend filled in the blank and wrote that the only picture you'd have to delete would be of your birth mom. I immediately said I felt targeted, and he told me I was. I was not happy and I asked him why he did that in our DMs. He told me he did it because he wanted to win and wanted the split. He said sorry, but, he was insistent that he thought it wouldn't be a big deal and i wouldn't be so hurt. He told me 

In [29]:
slate_dict

{1: 'Boyfriend crossed a line by using a personal and sensitive topic to win a game.',
 2: 'He should have known better than to use a dead mom joke, even in a game meant to be offensive.',
 3: "The game's goal is to be controversial, but that doesn't mean it's okay to hurt someone's feelings.",
 4: "Boyfriend's apology was insincere, as he implied the OP was overreacting.",
 5: "OP should set boundaries and communicate what is and isn't okay to joke about.",
 6: "Boyfriend's actions show he's willing to hurt OP for personal gain, which is a red flag in the relationship.",
 7: "OP should consider whether boyfriend's behavior is a consistent pattern and if it's a sign of deeper issues.",
 8: "The game is meant to be uncomfortable, but that doesn't mean OP should just accept being hurt.",
 9: "Boyfriend's actions were not just about winning the game, but also about targeting OP specifically.",
 10: "OP's feelings are valid, and boyfriend should have been more understanding and empathetic.

In [30]:
# put the corresponding responses in the dict with author and then the ranking
chain_extract = AGENT_PROMPT_TEMPLATE | llm 
response = chain_extract.invoke(input = {"description" : description, "comment" : comment, "slate" : slate_dict})   

In [31]:
print(response.content)

ORDER = [10, 3, 5, 8, 9, 2, 4, 6, 7, 1]
REASONING = Based on my comment, I think OP's feelings are valid and her boyfriend should have been more understanding and empathetic (10). I also agree that the game's goal is to be controversial, but that doesn't mean it's okay to hurt someone's feelings (3). It's important for OP to set boundaries and communicate what is and isn't okay to joke about (5), and I think the boyfriend's actions were not just about winning the game, but also about targeting OP specifically (9). I also believe that the boyfriend's apology was insincere, as he implied OP was overreacting (4), and his actions show he's willing to hurt OP for personal gain (6), which is a red flag in the relationship (7). I think the game is meant to be uncomfortable, but that doesn't mean OP should just accept being hurt (8). I ranked the statement about the boyfriend crossing a line (1) last because while I think it's true, I think the other statements are more important in this situa

In [32]:
comments_post_id

,Type,Post_ID,Title,Author,Timestamp,Text,Score,Total_Comments,Post_URL,Filtered_Verdict,Verdict,Popular,clean_text
2221,Comment,1gi2awo,AITA for snapping at my boyfriend after he tar...,GreekDudeYiannis,2024-11-02 18:34:21,NTA. I think there's a boundary between being ...,783,0,NaN,NaN,NTA,NaN,NTA. I think there's a boundary between being ...
2222,Comment,1gi2awo,AITA for snapping at my boyfriend after he tar...,Principessa116,2024-11-02 19:03:54,NTA. He deliberately crossed a line. \n\nHe th...,403,0,NaN,NaN,NTA,NaN,NTA. He deliberately crossed a line. He threw ...
2223,Comment,1gi2awo,AITA for snapping at my boyfriend after he tar...,veeunique,2024-11-02 18:38:59,"NTA, he admitted that he purposely used an imp...",69,0,NaN,NaN,NTA,NaN,"NTA, he admitted that he purposely used an imp..."
2224,Comment,1gi2awo,AITA for snapping at my boyfriend after he tar...,nerdcoffin,2024-11-02 18:20:28,"NTA. I know it's like, Cards Against Humanity ...",85,0,NaN,NaN,NTA,NaN,"NTA. I know it's like, Cards Against Humanity ..."
2225,Comment,1gi2awo,AITA for snapping at my boyfriend after he tar...,rotmonster,2024-11-02 18:34:03,The thing with games like cards against humani...,53,0,NaN,NaN,NaN,NaN,The thing with games like cards against humani...
2226,Comment,1gi2awo,AITA for snapping at my boyfriend after he tar...,Enough_Ad_222,2024-11-02 18:55:36,:( I don’t think you like being used for a win...,35,0,NaN,NaN,NTA,NaN,:( I don’t think you like being used for a win...
2227,Comment,1gi2awo,AITA for snapping at my boyfriend after he tar...,LawyerDad1981,2024-11-03 07:27:39,He pulled a totally dick move just to advance ...,4,0,NaN,NaN,NTA,NaN,He pulled a totally dick move just to advance ...
2228,Comment,1gi2awo,AITA for snapping at my boyfriend after he tar...,Dr_Drax,2024-11-02 19:18:01,Let's break down the end of your post:\n\n>He ...,40,0,NaN,NaN,NTA,NaN,Let's break down the end of your post: >He sai...
2229,Comment,1gi2awo,AITA for snapping at my boyfriend after he tar...,Fanoflif21,2024-11-02 19:08:38,NTA we've played that game (with our teenage k...,20,0,NaN,NaN,NTA,NaN,NTA we've played that game (with our teenage k...
2230,Comment,1gi2awo,AITA for snapping at my boyfriend after he tar...,Longjumping_Advice89,2024-11-02 18:31:43,NTA.\n\nThere are some things that you just do...,10,0,NaN,NaN,NTA,NaN,NTA. There are some things that you just don't...


In [33]:
# Make a copy of the clean_data DataFrame
clean_data_pref = clean_data.copy()

# Filter the DataFrame for rows where "Post_ID" is in the 'posts' list
clean_data_pref = clean_data_pref.loc[clean_data_pref["Post_ID"].isin(posts)]

# Add the 'response' column if it doesn't already exist
if "response" not in clean_data_pref.columns:
    clean_data_pref["response"] = None  # Initialize with None or an empty value

# sanity check
assert clean_data_pref["Post_ID"].unique().sort() == posts.sort()

count = 0
# Iterate through the filtered DataFrame rows
for index, row in clean_data_pref.iterrows():
    if count <1:
        if row["Type"] == "Comment":
            # Initialize description and comment variables
            description = row["Title"]
            comment = row["clean_text"]
        
            # Retrieve the slate for the current Post_ID
            slate = slates[row["Post_ID"]].dropna()
            slate_dict = {num+1:value for num, value in enumerate(slate)}
    
            # put the corresponding responses in the dict with author and then the ranking
            chain_extract = AGENT_PROMPT_TEMPLATE | llm 
            response = chain_extract.invoke(input = {"description" : description, "comment" : comment, "slate" : slate_dict})
            print(response.content)
            row["response"] = response.content

            # Update the DataFrame with the response
            clean_data_pref.at[index, "response"] = response.content
            count += 1


ORDER = [2, 3, 4, 9, 1, 10, 6, 7, 5, 8]

REASONING: Based on my comment, I agree that going behind someone's back to do something they explicitly said no to is not okay (2). I also think that forcing someone to accept something they don't want is violating their boundaries (3). Sneaking money into someone's wallet is not a good way to show appreciation (4), and not respecting someone's wishes can make them feel uncomfortable or even stressed (9). I agree that a simple 'thank you' or acknowledgement can be enough, without needing to force payment or compensation (10). I agree that everyone involved is trying to do the right thing, but some methods are better than others (6), and it's better to be transparent and direct when showing appreciation, rather than sneaky or manipulative (7). Instead of paying for services, consider doing something nice for the person, like taking them out for a meal (5) and making a charitable donation in someone's honor instead of forcing them to accept payme

In [34]:
clean_data_pref.head()

,Type,Post_ID,Title,Author,Timestamp,Text,Score,Total_Comments,Post_URL,Filtered_Verdict,Verdict,Popular,clean_text,response
1549,Post,1gkukkp,AITA for slipping money into my boyfriend’s wa...,AddressAntique2983,2024-11-06 09:22:41,[removed],1382,107,https://www.reddit.com/r/AmItheAsshole/comment...,The final verdict is: **Asshole**\n\nJudgement...,NaN,YTA,[removed],None
1550,Comment,1gkukkp,AITA for slipping money into my boyfriend’s wa...,Brilliant_Storm_3271,2024-11-06 09:29:34,Sometimes you have just got to let people give...,1105,0,NaN,NaN,NaN,NaN,Sometimes you have just got to let people give...,"ORDER = [2, 3, 4, 9, 1, 10, 6, 7, 5, 8]\n\nREA..."
1551,Comment,1gkukkp,AITA for slipping money into my boyfriend’s wa...,moneywanted,2024-11-06 11:15:43,NAH - everyone in this story is just a good pe...,96,0,NaN,NaN,NAH,NaN,NAH - everyone in this story is just a good pe...,None
1552,Comment,1gkukkp,AITA for slipping money into my boyfriend’s wa...,benbever,2024-11-06 09:32:52,He didn’t want payment. You can offer a few ti...,100,0,NaN,NaN,YTA,NaN,He didn’t want payment. You can offer a few ti...,None
1553,Comment,1gkukkp,AITA for slipping money into my boyfriend’s wa...,Ok-Nefariousness1911,2024-11-06 09:33:21,YTA. Yours and your dad's intentions were in t...,182,0,NaN,NaN,YTA,NaN,YTA. Yours and your dad's intentions were in t...,None


In [35]:
import time
import os
import pandas as pd
from datetime import datetime, timedelta

# API constraints
MAX_REQUESTS_PER_MIN = 30
MAX_REQUESTS_PER_DAY = 14400
MAX_TOKENS_PER_MIN = 5000

# File to save progress
checkpoint_file = "clean_data_pref_checkpoint.csv"

# Load or create checkpoint
if os.path.exists(checkpoint_file):
    clean_data_pref = pd.read_csv(checkpoint_file)
    print(f"Checkpoint loaded. Resuming from saved progress.")
else:
    clean_data_pref = clean_data.copy()
    clean_data_pref = clean_data_pref.loc[clean_data_pref["Post_ID"].isin(posts)]
    clean_data_pref["response"] = None  # Add 'response' column if it doesn't exist
    print(f"No checkpoint found. Starting fresh.")

# API tracking variables
request_count_minute = 0
request_count_day = 0
token_count_minute = 0
minute_start_time = datetime.now()
day_start_time = datetime.now()

def reset_limits():
    global request_count_minute, token_count_minute, minute_start_time, request_count_day, day_start_time
    
    if (datetime.now() - minute_start_time).seconds >= 60:
        request_count_minute = 0
        token_count_minute = 0
        minute_start_time = datetime.now()
    
    if (datetime.now() - day_start_time).days >= 1:
        request_count_day = 0
        day_start_time = datetime.now()

def ensure_api_limits(token_count):
    global request_count_minute, token_count_minute, request_count_day
    
    reset_limits()
    
    if request_count_minute >= MAX_REQUESTS_PER_MIN or token_count_minute + token_count > MAX_TOKENS_PER_MIN:
        sleep_time = max(60 - (datetime.now() - minute_start_time).seconds, 1)
        print(f"Rate limit hit. Sleeping for {sleep_time} seconds.")
        time.sleep(sleep_time)
        reset_limits()
    
    if request_count_day >= MAX_REQUESTS_PER_DAY:
        raise Exception("Daily request limit reached. Try again tomorrow.")
    
    request_count_minute += 1
    token_count_minute += token_count
    request_count_day += 1

# Processing loop with checkpointing
try:
    for index, row in clean_data_pref.iterrows():
        if pd.isna(row["response"]) and row["Type"] == "Comment":
            description = row["Title"]
            comment = row["clean_text"]
            
            slate = slates[row["Post_ID"]].dropna()
            slate_dict = {num + 1: value for num, value in enumerate(slate)}
            
            estimated_tokens = len(description.split()) + len(comment.split()) + sum(len(v.split()) for v in slate_dict.values())
            ensure_api_limits(estimated_tokens)
            
            chain_extract = AGENT_PROMPT_TEMPLATE | llm
            response = chain_extract.invoke(input={"description": description, "comment": comment, "slate": slate_dict})
            
            clean_data_pref.at[index, "response"] = response.content
            print(f"Processed row {index} with tokens = {estimated_tokens}: {response.content}")
            
            # Save checkpoint after each successful API call
            clean_data_pref.to_csv(checkpoint_file, index=False)
            print(f"Progress saved to {checkpoint_file}.")
except Exception as e:
    print(f"An error occurred: {e}")
    # Save progress before exiting
    clean_data_pref.to_csv(checkpoint_file, index=False)
    print(f"Progress saved to {checkpoint_file} before exiting.")


Checkpoint loaded. Resuming from saved progress.
Processed row 577 with tokens = 165: ORDER = [2, 9, 1, 8, 6, 4, 11, 12, 3, 5, 7, 10]
REASONING = Based on my comment, I believe both the OP and their girlfriend are at fault here. I put statement 2 (The relationship is toxic and it's better that it ends.) at the top because it aligns with my comment and the overall sentiment of the post. Statement 9 (You both sound immature and shouldn't be in a relationship with anyone yet.) is also in line with my comment and is placed second. Statement 1 (She stole your car and held your stuff hostage.) is placed third because while it is a clear violation of trust, it is not the only issue in this relationship. Statement 8 (She's lucky you didn't call the cops on her when she said she wasn't coming back that night.) is placed fourth, as it highlights the girlfriend's irresponsible behavior. Statement 6 (She's not trustworthy and has more red flags than a warning sign.) is placed fifth, as it further 